In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
dataset1 = tf.data.TextLineDataset('data/2019-10-04_12-03-39.mp4-57.mp4.txt')
dataset2 = tf.data.TextLineDataset('data/2019-10-04_12-03-39.mp4-78.mp4.txt')

In [3]:
## Format Sequence of keypoints
data = []
lengths = []
for line in dataset1.take(10000):
    l = line.numpy().split(' ')
    if len(l) == 59:
        data.append(l)
    
files = [d[0] for d in data]
a = [ np.array((d[25::2],d[26::2]), dtype=float).transpose() for d in data]

# translate keypoints based on  nose position (origin = nose at first frame)
origin = a[0][0]
a = a - origin
np.shape(a)

(24, 17, 2)

In [4]:
## Format Sequence of keypoints
data = []
lengths = []
for line in dataset2.take(10000):
    l = line.numpy().split(' ')
    if len(l) == 59:
        data.append(l)
    
files = [d[0] for d in data]
b = [ np.array((d[25::2],d[26::2]), dtype=float).transpose() for d in data]

# translate keypoints based on  nose position (origin = nose at first frame)
origin = b[0][0]
b = b - origin
np.shape(b)

(24, 17, 2)

In [5]:
print(np.shape(a), np.shape(b))

((24, 17, 2), (24, 17, 2))


In [14]:
# np.linalg.norm(a-b)

norm = lambda x: np.linalg.norm(x)

def distance(_a,_b): 
    
    ctc = 0
    for i in range(len(_a)):
        dab_min = np.inf
        k = 0
        for j in range(len(_b)):
#             dab = np.linalg.norm(xa-xb)
            dab = np.sum(map(norm, (_a[i]-_b[j])))           
            if dab == dab_min and abs(j-i)<abs(k-i):                
                k = j
            elif dab < dab_min:
                dab_min = dab
                k = j

        # "k" is the index of the element in "b", closest to the current element of "a"
        # "b[k]" is the cosest element from "b" to the current element of "a" 
        dba_min = np.inf
        n = 0
        for j in range(len(_a)):
#             dba = np.linalg.norm(xaa-xbb)
            dba = np.sum(map(norm, (_a[j]-_b[k])))  
            if dba == dba_min and abs(j-k)<abs(n-k):                
                n = j
            elif(dba < dba_min):
                dba_min = dba
                n = j
        ## if returned to the same element, ai should be equal to i
        ctc += abs(i-n)
        
#         print(i,k,n)

    return ctc

In [15]:
distance(b,a)

103